In [ ]:
import requests
import pandas as pd
def get_index():
    url = f"https://api.alternative.me/fng/"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        print(f"data is type: {type(data)}")
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error getting fear&greed data: {e}")
        return None

In [2]:
index_data = get_index()
index_data

{'name': 'Fear and Greed Index',
 'data': [{'value': '64',
   'value_classification': 'Greed',
   'timestamp': '1759363200',
   'time_until_update': '51225'}],
 'metadata': {'error': None}}

In [ ]:
from datetime import datetime

def figure_index(data):
    if not data or 'data' not in data:
        print("No data to clean")
        return None
    try:
        data_list = data['data']
        df = pd.DataFrame(data_list)
        df['timestamp'] = pd.to_numeric(df['timestamp'], errors = 'coerce')
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit = 's')
        df = df.drop(columns=['time_until_update'])
        df = df.rename(columns = {'value': 'value', 'value_classification': 'index', 'timestamp': 'date'})
        df = df[['date', 'index', 'value']]
        print(f"data is type: {type(df)}")
        return df
    except Exception as e:
        print(f"Error cleaning fear&greed data: {e}")
        return None

In [ ]:
index = figure_index(index_data)
index

,date,index,value
0,2025-10-02,Greed,64
